In [1]:
import torch
x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

In [2]:
# Can also create x = torch.arange(4.0, requires_grad=True)
x.requires_grad_(True)
x.grad  # Initially, the gradient is None by default. Only after running 'y.backward()' below it will be calculated.

In [3]:
y = 2 * torch.dot(x, x)
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
y.backward()
x.grad
#O parâmetro requires_grad=True serve para dizer ao PyTorch que ele deve acompanhar as operações realizadas no tensor x, construindo um grafo
#computacional que será usado para calcular os gradientes. Quando você chama y.backward(), o PyTorch percorre esse grafo e calcula as derivadas
#parciais de y em relação a todos os tensores que foram marcados com requires_grad=True (no caso, o tensor x).
#Se x não tivesse requires_grad=True, o PyTorch não rastrearia as operações realizadas com x e, consequentemente, não calcularia
#nem armazenaria o gradiente para x.
#Portanto, ao definir x=torch.arange(4.0) com requires_grad=True, você está dizendo que deseja que x seja um parâmetro
#"diferenciável", e é por isso que o y.backward() utiliza esse tensor para calcular e armazenar o gradiente em x.grad.
#Além disso, O atributo grad_fn=<MulBackward0> indica que o tensor foi criado a partir de uma operação diferenciável
#(neste caso, uma multiplicação) e que possui uma função associada para calcular o gradiente durante o processo de backpropagation.
#Ou seja, grad_fn: É um atributo interno que aponta para a "função de retropropagação" (backward function) que foi utilizada para gerar aquele tensor.
#E <MulBackward0>: Especifica que a operação que produziu o tensor foi uma multiplicação. Essa função será chamada quando você executar y.backward(),
#permitindo que o PyTorch calcule como cada operação (neste caso, a multiplicação) contribui para o gradiente final.
#Portanto, mesmo que o resultado numérico seja 28, o PyTorch mantém o histórico das operações (a árvore computacional) e, ao ver grad_fn=<MulBackward0>,
#sabe exatamente qual função usar para propagar o gradiente através dessa multiplicação.
#Caso requires_grad=True não tivesse sido utilizado, as operações não seriam rastreadas, x.grad não retornaria resultado, e o atributo grad_fn=<MulBackward0>
#não estaráia presente no resultado do cálculo de y

tensor([ 0.,  4.,  8., 12.])

In [5]:
x.grad == 4 * x #lembre que x.grad retorna o gradiente >de y< 
#o produto interno de dois vetores( torch.dot(x, x) ) é igual à norma desse vetor ao quadrado. O gradiente desta norma ao quadrado
#é igual a duas vezes este vetor. Portanto, 2 * torch.dot(x, x) == 4*x 

tensor([True, True, True, True])

In [6]:
x.grad.zero_()  # Reseta os valores do gradiente de y que haviam sido armazenados com x.requires_grad_(True) após rodar y.backward()
y = x.sum() #y agora é a função do somatório de todos os elementos do vetor x
y.backward() #calcula o gradiente de y em relação a x (em relação aos x_0, x_1, x_2 e x_3 presente em x)
y, x.grad # resultado da função y quando x=tensor([0., 1., 2., 3.]), e gradiente de y para valores de x
          #note que a derivada em relação a x_0 de x_0 é 1, de x_1 em relação a x_1 é 1, e assim por diante.

(tensor(6., grad_fn=<SumBackward0>), tensor([1., 1., 1., 1.]))

In [11]:
x.grad.zero_()
y = x * x 
y #agora, y não é mais um esvalar, e sim um vetor. Quando y, é vetor, y.backward() retorna erro. Para usá-lo, precisa transformá-lo em escalar

tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>)

In [12]:
y.backward(gradient=torch.ones(len(y)))  #Faster: y.sum().backward() #gradient=torch.ones(len(y)) transforma y em um escalar l
                                         #fazendo o produto interno entre y e um vetor v feito de uns (1). O gradiente é calculado
                                         #a partir deste resultado l
x.grad

tensor([0., 2., 4., 6.])

In [16]:
#If you simply calculate z and then z.backward(), PyTorch's automatic differentiation will trace back all dependencies to find the gradient
#of z with respect to x.  This gradient will reflect both the direct effect of x on z and the indirect effect channeled through y.
#Mathematically, using the chain rule, the total derivative dz/dx would consider both paths.

#you might want to know: "If I change x slightly, how much does z change directly, holding the value of y effectively constant at its current value?"
#This is where "detaching" comes in.  You want to treat y (or a value derived from y) as if it were a constant with respect to x when calculating
#the gradient of z with respect to x.
x.grad.zero_()
y = x * x

#Assume we want to calculate z = y * x

u = y.detach() #This is the crucial step. detach() does the following: Creates a new tensor u: u will have the same numerical value as y at the moment
               #of detachment. Breaks the computation graph link: Crucially, u is no longer considered part of the computational path that leads back
               #to x for gradient calculation. PyTorch "forgets" how u was derived from y and, ultimately, from x. u is treated as a constant with respect
               #to x in the backward pass.
z = u * x #Now, z is computed using x and the detached u, which will be treated as a constant
z.sum().backward() #Using basic calculus, the derivative of x * u with respect to x (where u is constant) is simply u (d/dx ux = u). 
x.grad == u #Remember that x.grad contains the stored result of the gradient calculated with z.sum().backward(), which is equal to
#z.backward(gradient=torch.ones(len(z))). And this gradient is, as shown above, d/dx ux = u. Therefore, x.grad == u

tensor([True, True, True, True])

In [17]:
#Note that while this procedure detaches y’s ancestors from the graph leading to z, the computational graph leading to y persists and thus we
#can calculate the gradient of y with respect to x. Como y = x^2, então d/dx y = 2x
x.grad.zero_()
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

In [20]:
def f(a):
    b = a * 2
    while b.norm() < 1000:
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

#A função acima retorna um valor linear para um valor de a. Ao tirar o gradiente de uma função linear, ele será igual ao resultado da função dividido pelo
#pelo próprio a. Por exemplo, se def f(a) retornasse 5*a, o grad em função de a é d/da 5a = 5. Se dividirmos 5a por a, também teremos 5.

a = torch.randn(size=(), requires_grad=True) #size=() gera um tensor de dimensão 0, ou seja, um escalar
d = f(a)
d.backward()
a.grad == d / a # portanto, se d = f(a), então a.grad é igual a d/a

tensor(True)